In [1]:
import os
from tqdm import tqdm
from itertools import combinations

from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import torch
import wandb
from pytorch_lightning import loggers
import netrd

import utils

In [2]:
if not os.path.isdir("./data"):
    !mkdir data

filename = "./data/proteins.npz"
if os.path.isfile(filename):
    labels, dist_mat = np.load(filename).values()
    N = len(labels)
else:
    proteins, labels = utils.read_graph_dataset("PROTEINS")
    N = len(proteins)
    
    ipsen_mikhailov_distance = netrd.distance.IpsenMikhailov()
    dist_mat = np.zeros((N, N), dtype=float)
    for i, j in tqdm(combinations(range(N), 2), total=N * (N - 1) // 2):
        dist_mat[i, j] = dist_mat[j, i] = ipsen_mikhailov_distance(proteins[i], proteins[j])
            
    np.savez(filename, labels=labels, dist_mat=dist_mat)

In [3]:
metrics = {
    "train_acc": [],
    "eval_acc": [],
    "eval_acc_bayes": [],
}

k_fold = KFold(n_splits=5, shuffle=True, random_state=42)
for idx_train, idx_eval in k_fold.split(torch.arange(N)):
    labels_train, labels_eval = labels[idx_train], labels[idx_eval]
    dist_mat_train, dist_mat_eval = dist_mat[idx_train[:, None], idx_train], dist_mat[idx_train[:, None], idx_eval]
    
    bmds = utils.SklearnBMDS(bmds_train_kwargs={"max_dim": 40, "threshold": 0.1, "n_iters_check": 20}, batch_size_train=1e-1)
    
    run = wandb.init(project="proteins bmds train")
    logger = loggers.WandbLogger()
    x_train, std_train = bmds.fit_transform(dist_mat_train, max_epochs=100, logger=logger)
    run.finish()
    
    run = wandb.init(project="proteins bmds eval")
    logger = loggers.WandbLogger()
    x_eval, std_eval = bmds.transform(dist_mat_eval, logger=logger, max_epochs=100)
    run.finish()
    
    clf = utils.SklearnMLPClassifier(labels_train.max() + 1)
    run = wandb.init(project="proteins bmds clf")
    logger = loggers.WandbLogger()
    pred_train = clf.fit_predict(x_train, labels_train, std_train, logger=logger, max_epochs=200)
    pred_eval = clf.predict(x_eval)
    pred_eval_bayes = clf.predict(x_eval, std_eval)
    logger.log_hyperparams({
        "train_acc": accuracy_score(pred_train, labels_train),
        "eval_acc": accuracy_score(pred_eval, labels_eval),
        "eval_acc_bayes": accuracy_score(pred_eval_bayes, labels_eval),
    })
    run.finish()
    
    metrics["train_acc"].append(accuracy_score(pred_train, labels_train))
    metrics["eval_acc"].append(accuracy_score(pred_eval, labels_eval))
    metrics["eval_acc_bayes"].append(accuracy_score(pred_eval_bayes, labels_eval))

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: antonii-belyshev (ai-prentice). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.8/dist-packages/pytorch_lightning/loggers/wandb.py:347: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type | Params
------------------------------
------------------------------
0         Trainable params
0         Non-trainable params
0         Total params
0.000     Total estimated model params size (MB)



Learning the optimal train embedding...


/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/trainer.py:1933: PossibleUserWarning: The number of training batches (11) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

...

dim,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,█▅▂▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
reg,▁▃▅▇▇▇██████████████████████████████████
total_loss,█▅▂▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
dim,5.0
epoch,99
loss,11.16955
reg,8.8351
total_loss,20.00465


/usr/local/lib/python3.8/dist-packages/pytorch_lightning/loggers/wandb.py:347: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type             | Params
---------------------------------------------
0 | layers  | ModuleList       | 385   
1 | loss_fn | CrossEntropyLoss | 0     
---------------------------------------------
385       Trainable params
0         Non-trainable params
385       Total params
0.002     Total estimated model params size (MB)
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have man

Training: 0it [00:00, ?it/s]

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,▆▅▅▆▄▅▆▅▆▅▃▅▂▇▃█▇▅▄▄▃▂▄▆▅▃▄▇▂▁▅▂▂▅▂▆▂▅▃▃
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch,199
loss,0.38134
trainer/global_step,22399


In [4]:
metrics_df = pd.DataFrame(metrics)
metrics_df

,train_acc,eval_acc,eval_acc_bayes
0,0.800000,0.748879,0.757848
1,0.730337,0.784753,0.784753
2,0.774157,0.726457,0.735426
3,0.785634,0.806306,0.810811
4,0.792368,0.765766,0.765766


In [5]:
mean_values = metrics_df.mean()
std_values = metrics_df.std()

summary_df = pd.DataFrame([mean_values, std_values], index=['Mean', 'Std'])
summary_df

,train_acc,eval_acc,eval_acc_bayes
Mean,0.776499,0.766432,0.770921
Std,0.027495,0.030952,0.028460
